<a href="https://colab.research.google.com/github/bhas10bc/Deep_learning/blob/main/loan_default_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [102]:
data = pd.read_csv('Loan_Default.csv')

In [103]:
data.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [104]:
data.fillna(data.mean(), inplace= True)

<ipython-input-104-e9f80281d5f3>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.fillna(data.mean(), inplace= True)


In [105]:

# Define the categorical columns to be one-hot encoded
categorical_columns = [
    'loan_limit',
    'Gender',
    'approv_in_adv',
    'loan_type',
    'open_credit',
    'loan_purpose',
    'business_or_commercial',
    'Neg_ammortization',
    'interest_only',
    'lump_sum_payment',
    'construction_type',
    'occupancy_type',
    "total_units",
    'Secured_by',
    'credit_type',
    'co-applicant_credit_type',
    'submission_of_application',
    'Region',
    'Security_Type',
    'Status',
]

In [106]:
data = pd.get_dummies(data, columns = categorical_columns, drop_first=True)

In [107]:
age_range_mapping = {
    '<25': 25,
    '25-34': 30,
    '35-44': 40,
    '45-54': 50,
    '55-64': 60,
    '65-74': 70,
    '>74': 75  # You can choose an appropriate value for '>74'
}

In [108]:
data['age'] = data['age'].map(age_range_mapping)

In [109]:
X = data.drop('Credit_Worthiness', axis=1)
y = data['Credit_Worthiness']

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [111]:
scaler = StandardScaler()

In [112]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [113]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder
label_encoder = LabelEncoder()

# Fit and transform the target variable for the training data
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform the target variable for the test data (do not fit again)
y_test_encoded = label_encoder.transform(y_test)

In [114]:
import tensorflow as tf
from tensorflow import keras

In [115]:
model = keras.Sequential([
    keras.layers.Input(shape = X_train.shape[1]),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid'),
])

In [116]:
model.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [117]:
from keras.src.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train_encoded, epochs =100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [120]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)

930/930 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9589


In [121]:
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.9589022397994995
